In [2]:
import numpy as np 
import pandas as pd
import os, zipfile

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image

from keras.preprocessing.image import load_img, img_to_array
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


<h1>Preprocessing</h1>

In [3]:
# read in data
train_df = pd.read_csv('./train_onelabel.csv')
train_images_paths = ['./train_images/' + p for p in train_df.image]
train_labels = train_df['class'].values

In [4]:
# create image matrix
train_images = (load_img(p, target_size=(32, 32)) for p in train_images_paths)
train_images = [img_to_array(img)[:, :, [0]] for img in train_images]
train_images = np.array(train_images)
train_images.shape

(24204, 32, 32, 1)

In [5]:
# preprocessing of labels
classes = len(np.unique(train_labels))
train_labels = to_categorical(train_labels, classes)

In [6]:
# split data into test and train set
x_train, x_val, y_train,  y_val = train_test_split(train_images, train_labels, test_size=0.2)

In [7]:
# normalize image data

x_train = x_train.astype('float32')
x_val = x_val.astype('float32')
x_train /= 255
x_val /= 255

<h1>Model</h1>

In [8]:
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout

In [9]:
# Initialising the CNN
classifier = Sequential()

In [10]:
# Step 1 - Convolution ---TODO: see input specification
classifier.add(Conv2D(64, (3, 3), input_shape = (32, 32,1),padding='same', activation = 'relu'))

In [11]:
# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))

In [12]:
# Adding a second convolutional layer+pooling layer
classifier.add(Conv2D(64, (3, 3), activation = 'relu', padding='same'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

In [13]:
# Step 3 - Flattening
classifier.add(Flatten())

In [14]:
#Step 4 - Create a fully connected neural network
'''its good practise to pick a number of power of two-experimenting output_dim
128 hidden nodes in hidden layers-by experimenting'''
 

#output_dim = number of nodes in the hidden layer
classifier.add(Dense(units= 256, activation = 'relu'))

#Dropout (p was used in the old API, changed it to the new standard)
classifier.add(Dropout(rate = 0.2))

#second hidden layer
classifier.add(Dense(units = 256, activation = 'relu'))

#Dropout
#classifier.add(Dropout(p = 0.2))

#output layer
classifier.add(Dense(units = 121, activation = 'softmax')) #We have 121 categories of plankton!!

In [15]:
# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [16]:
# fit model on training data
classifier.fit(x_train, y_train, batch_size=80, nb_epoch=25, verbose=1)

C:\Users\movetech\Anaconda2\envs\py35\lib\site-packages\keras\models.py:874: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/25
19363/19363 [==============================] - 99s 5ms/step - loss: 3.5566 - acc: 0.1820
Epoch 2/25
19363/19363 [==============================] - 98s 5ms/step - loss: 2.5551 - acc: 0.3369
Epoch 3/25
19363/19363 [==============================] - 95s 5ms/step - loss: 2.2233 - acc: 0.3972
Epoch 4/25
19363/19363 [==============================] - 99s 5ms/step - loss: 2.0246 - acc: 0.4331
Epoch 5/25
19363/19363 [==============================] - 94s 5ms/step - loss: 1.9059 - acc: 0.4640
Epoch 6/25
19363/19363 [==============================] - 94s 5ms/step - loss: 1.7918 - acc: 0.4826
Epoch 7/25
19363/19363 [==============================] - 90s 5ms/step - loss: 1.7045 - acc: 0.5065
Epoch 8/25
19363/19363 [==============================] - 95s 5ms/step - loss: 1.6264 - acc: 0.5248
Epoch 9/25
19363/19363 [==============================] - 93s 5ms/step - loss: 1.5562 - acc: 0.5412
Epoch 10/25
19363/19363 [==============================] - 85s 4ms/step - loss: 1.4855 - acc: 0.5618

In [17]:
score = classifier.evaluate(x_val, y_val, verbose=0)

In [18]:
score

[1.5041827794639218, 0.60173517871902782]

In [17]:
score

[1.6067611749840531, 0.55009295603776365]

<h1>Save Model</h1>

In [18]:
from keras.models import model_from_json

# serialize model to JSON
best_model_classification_json = classifier.to_json()
with open("best_model_4.json", "w") as json_file:
    json_file.write(best_model_classification_json)
# serialize weights to HDF5
classifier.save_weights("model_best.h5")
print("Saved model to disk")

Saved model to disk


<h1>Load Model</h1>

In [12]:
from keras.models import model_from_json

# load json and create model
json_file = open('best_model_3.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
classifier = model_from_json(loaded_model_json)
# load weights into new model
classifier.load_weights("model_best.h5")
print("Loaded model from disk")

Loaded model from disk


<h1>Prediction - First Approach</h1>

In [21]:
# read in data
test_df = pd.read_csv('./sample.csv')
test_images_paths = ['./test_images/' + p for p in test_df['image']]
test_images_names = [p for p in test_df['image']]

In [22]:
# create image matrix
test_images = (load_img(p, target_size=(32, 32)) for p in test_images_paths)
test_images = [img_to_array(img)[:, :, [0]] for img in test_images]
test_images = np.array(test_images)
test_images.shape

(6132, 32, 32, 1)

In [23]:
# normalize image data
test_images = test_images.astype('float32')
test_images /= 255

In [24]:
result = classifier.predict(test_images)

In [25]:
labels = []

for count, i in enumerate(result):
    labels.append(np.argmax(result[count]))

In [41]:
from collections import OrderedDict

#Create Final csv for the competition
csv_list =  pd.DataFrame( OrderedDict( { 'image': pd.Series(test_images_names), 'class': pd.Series(labels)} ) )
csv_list.to_csv('sample4.csv',index=False)

In [42]:
results1 = pd.read_csv('./sample4.csv')

In [43]:
results1

,class,image
0,77,90715.jpg
1,117,159631.jpg
2,34,4294.jpg
3,3,56548.jpg
4,13,120979.jpg
5,101,139460.jpg
6,0,78510.jpg
7,42,102841.jpg
8,31,84208.jpg
9,53,9591.jpg
